# Inference script for Model 2
    1. Loading necessary data
    2. Initializing model
    3. Calculate continuous malignancy score based on mitotic count

## 1. Loading necessary data

In [ ]:
%load_ext autoreload
%autoreload 2

import pickle
import numpy as np
import WHO_REG_nn

model_file = "models/Model2_seed2.pth"
annotation_dict = pickle.load(open("wsis/test_set/MC_and_ROI_90.p","rb"))

In [2]:
import csv
# Annotation file with who grades
csv_file = ""
who_grades = dict()
with open(csv_file, newline = '')as file:
    reader = csv.reader(file, delimiter = ',')
    for idx,line in enumerate(reader):
        if idx == 0:
            continue
        who_grades[line[0]] = list()
        who_grades[line[0]].extend([int(line[1]),int(line[2])])

In [3]:
new_annotation = dict()

for key in annotation_dict:
    for sub_key in annotation_dict[key]:
        new_annotation[sub_key]=dict()
        new_annotation[sub_key]['roi'] = annotation_dict[key][sub_key][1]
        new_annotation[sub_key]['mitotic_count'] = annotation_dict[key][sub_key][0]
        new_annotation[sub_key]['who_grade'] = who_grades['_'.join(sub_key.split(' ')[0].split('_')[0:2])][0]
annotation_dict = new_annotation

In [4]:
# Standardize mitosis count
mcs = [annotation_dict[key]['mitotic_count'] for key in annotation_dict]
grades = [annotation_dict[key]['who_grade'] for key in annotation_dict]

mean_mcs = 4.222873900293255
std_mcs = 6.724372788197874
max_mcs = 11.417737909245652

mcs = [(m-mean_mcs)/std_mcs for m in mcs]

mcs = [m/max_mcs for m in mcs]
for key,m in zip (annotation_dict, mcs):
    annotation_dict[key]['mitotic_count'] = m

## 2. Initializing model

In [5]:
import torch 
model = WHO_REG_nn.SimpleRegressionModel()
model.load_state_dict(torch.load(model_file))

<All keys matched successfully>

## 3. Calculate continuous malignancy score based on mitotic count

In [6]:
mcs = torch.tensor(mcs, dtype = torch.float32).reshape(-1,1)

with torch.no_grad():
    out1 = model(mcs)

/usr/local/lib/python3.8/dist-packages/torch/nn/functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [ ]:
file = ""

pickle.dump(out1,open(file,"wb"))